In [9]:
import pandas as pd
import numpy as np
import sklearn

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [11]:
%pip install uv



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
uv --version


uv 0.9.5
Note: you may need to restart the kernel to use updated packages.


In [13]:
!python3 --version


Python 3.12.1


In [15]:
import os
print(os.getcwd())


/workspaces/DataTalksMlZoomCamp/05-Deployment


In [17]:
import pickle
import pandas as pd

# load the pipeline
with open("/workspaces/DataTalksMlZoomCamp/homework/pipeline_v1.bin", "rb") as f:
    pipeline = pickle.load(f)

# record to score
record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

# use a DataFrame in case pipeline expects it
X = [record]

# get probability of conversion (usually positive class = 1)
proba = pipeline.predict_proba(X)[0, 1]

print("Probability of conversion:", proba)


Probability of conversion: 0.5336072702798061


In [1]:
!pip install fastapi uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [fastapi]5;237m━━━━ 8/9 [fastapi]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [5]:
import pickle
from fastapi import FastAPI
from pydantic import BaseModel

# 1. Load the pipeline
with open("pipeline_v1.bin", "rb") as f:
    pipeline = pickle.load(f)

# 2. Define input schema
class Client(BaseModel):
    lead_source: str
    number_of_courses_viewed: int
    annual_income: float

# 3. Create FastAPI app
app = FastAPI()

@app.get("/")
def root():
    return {"message": "Model API is running"}

@app.post("/predict")
def predict(client: Client):
    record = client.dict()
    X = [record]   # DictVectorizer expects list of dicts
    proba = pipeline.predict_proba(X)[0, 1]
    return {"subscription_probability": proba}


/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 1.7.0. Th

In [6]:
uvicorn main:app --reload --host 0.0.0.0 --port 8000


SyntaxError: invalid syntax (3774377372.py, line 1)

In [7]:
!uvicorn main:app --reload --host 0.0.0.0 --port 8000


INFO:     Will watch for changes in these directories: ['/workspaces/DataTalksMlZoomCamp/05-Deployment/homework']
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [17529] using StatReload
ERROR:    Error loading ASGI app. Attribute "app" not found in module "main".
INFO:     Started server process [46264]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [46264]
INFO:     Started server process [46397]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [8]:
import requests

url = "http://0.0.0.0:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())


{'subscription_probability': 0.5340417283801275, 'subscribed': True}


In [11]:
import requests

url = "http://localhost:8000/predict"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())


{'subscription_probability': 0.9933071490756734, 'subscribed': True}
